In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [ ]:




for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "Tweets.csv" with length 3421431 bytes


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.read_csv('Tweets.csv')

In [ ]:
df=df[["text","airline_sentiment"]]

In [ ]:

df

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
...,...,...
14635,@AmericanAir thank you we got on a different f...,positive
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative
14637,@AmericanAir Please bring American Airlines to...,neutral
14638,"@AmericanAir you have my money, you change my ...",negative


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = set(stopwords.words("english"))
print(stop_words)

{"you've", 'between', "isn't", 'now', 'how', 'have', 'up', 're', 'at', 'were', 'and', 'yourself', "mustn't", 'those', 'if', 'under', 'further', 'to', "hasn't", 'isn', 'into', 'off', 'him', 'after', 'hasn', "should've", 'against', 'o', 'her', 'with', 'its', 'theirs', 'me', 'most', 'more', 'shan', 'themselves', 'why', "mightn't", "haven't", 'but', "you're", "that'll", 'who', 'does', 'herself', 'any', 'nor', "you'll", 'mightn', 'as', 'shouldn', "couldn't", 'this', 'we', 'hers', 'while', "won't", 'of', 'had', 'itself', 'there', "shan't", "didn't", 'my', 'an', 'hadn', 'for', "don't", 'same', 'once', 'his', 'won', 'be', 'the', 'above', 'ours', 'weren', 'you', 'our', 'yourselves', 'myself', 'm', "you'd", 'are', 'each', 'himself', 'doesn', 'such', 't', "it's", 'll', 'has', 'do', 'only', "hadn't", 'them', 'because', 'before', 'will', 'i', 'what', 'that', 'where', "weren't", 'so', 'these', 'yours', 'was', 'some', 'too', 'other', 'wasn', 'needn', 'she', 'then', 's', 'couldn', 'through', 'd', "are

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
wordnet = WordNetLemmatizer()
def text_preproc(x):
  x = x.lower()
  x = ' '.join([word for word in x.split(' ') if word not in stop_words])
  x = x.encode('ascii', 'ignore').decode()
  x = re.sub(r'https*\S+', ' ', x)
  x = re.sub(r'@\S+', ' ', x)
  x = re.sub(r'#\S+', ' ', x)
  x = re.sub(r'\'\w+', '', x)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*', '', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x

In [ ]:
df['clean_text'] = df.text.apply(text_preproc)

In [ ]:
df

,text,airline_sentiment,clean_text
0,@VirginAmerica What @dhepburn said.,neutral,said
1,@VirginAmerica plus you've added commercials t...,positive,plus added commercials experience tacky
2,@VirginAmerica I didn't today... Must mean I n...,neutral,today must mean need take another trip
3,@VirginAmerica it's really aggressive to blast...,negative,really aggressive blast obnoxious entertainme...
4,@VirginAmerica and it's a really big bad thing...,negative,really big bad thing
...,...,...,...
14635,@AmericanAir thank you we got on a different f...,positive,thank got different flight chicago
14636,@AmericanAir leaving over 20 minutes Late Flig...,negative,leaving minutes late flight warnings communic...
14637,@AmericanAir Please bring American Airlines to...,neutral,please bring american airlines
14638,"@AmericanAir you have my money, you change my ...",negative,money change flight answer phones suggestions...


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["clean_text"],df["airline_sentiment"],test_size=0.1,shuffle=True)

In [ ]:

tfidf_vectorizer = TfidfVectorizer(use_idf=True,max_features=1500)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:

lr_tfidf=MultinomialNB()
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  

#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

              precision    recall  f1-score   support

    negative       0.74      0.97      0.84       941
     neutral       0.65      0.29      0.40       287
    positive       0.83      0.38      0.52       236

    accuracy                           0.74      1464
   macro avg       0.74      0.55      0.59      1464
weighted avg       0.74      0.74      0.70      1464

Confusion Matrix: [[912  22   7]
 [193  83  11]
 [124  22  90]]


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:

from sklearn.linear_model import LogisticRegression

In [ ]:

clf2 = LogisticRegression(solver = "lbfgs")

In [ ]:

clf2.fit(X_train_vectors_tfidf, y_train)  


LogisticRegression()

In [ ]:
y_predict = clf2.predict(X_test_vectors_tfidf)
y_prob = clf2.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

    negative       0.81      0.92      0.86       941
     neutral       0.62      0.50      0.55       287
    positive       0.80      0.53      0.64       236

    accuracy                           0.78      1464
   macro avg       0.74      0.65      0.68      1464
weighted avg       0.77      0.78      0.76      1464

